# Biopython notes

DNA and protein sequences are the most common data found in biopython with the standard FASTA format
- Begins with one line identifier beginning with `>` with sequence following

```
> SEQ 1
MTEITAAMVKEL

> SEQ 2
TAAMVEITAAMV
```

## Biopython SeqIO package
- provides interface for working with sequence formats

`parse(file_path, format)` -  read in sequence files as SeqRecord object which contains the following info
- `id` ... ID used to identify the sequence -- a string
- `seq` ... convert to string: str(), check length of sequence with len()

In [2]:
from Bio import SeqIO

sequences = []
for seq_record in SeqIO.parse("/Users/fortino.pinedaveloz/VsCode/FPV_BIX/biopython/ls_orchid.fasta", "fasta"):
    # add record to list 
    sequences.append(str(seq_record.seq))
    # print sequence
    print(seq_record.seq)
    # print sequence identifier
    print(seq_record.id)
    # print len of sequence 
    print(len(seq_record))



CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGGAATAAACGATCGAGTGAATCCGGAGGACCGGTGTACTCAGCTCACCGGGGGCATTGCTCCCGTGGTGACCCTGATTTGTTGTTGGGCCGCCTCGGGAGCGTCCATGGCGGGTTTGAACCTCTAGCCCGGCGCAGTTTGGGCGCCAAGCCATATGAAAGCATCACCGGCGAATGGCATTGTCTTCCCCAAAACCCGGAGCGGCGGCGTGCTGTCGCGTGCCCAATGAATTTTGATGACTCTCGCAAACGGGAATCTTGGCTCTTTGCATCGGATGGAAGGACGCAGCGAAATGCGATAAGTGGTGTGAATTGCAAGATCCCGTGAACCATCGAGTCTTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCTTGGGCGTCGCGCTTCGTCTCTCTCCTGCCAATGCTTGCCCGGCATACAGCCAGGCCGGCGTGGTGCGGATGTGAAAGATTGGCCCCTTGTGCCTAGGTGCGGCGGGTCCAAGAGCTGGTGTTTTGATGGCCCGGAACCCGGCAAGAGGTGGACGGATGCTGGCAGCAGCTGCCGTGCGAATCCCCCATGTTGTCGTGCTTGTCGGACAGGCAGGAGAACCCTTCCGAACCCCAATGGAGGGCGGTTGACCGCCATTCGGATGTGACCCCAGGTCAGGCGGGGGCACCCGCTGAGTTTACGC
gi|2765658|emb|Z78533.1|CIZ78533
740
CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAGAATATATGATCGAGTGAATCTGGAGGACCTGTGGTAACTCAGCTCGTCGTGGCACTGCTTTTGTCGTGACCCTGCTTTGTTGTTGGGCCTCCTCAAGAGCTTTCATGGCAGGTTTGAACTTTAGTACGGTGCAGTTTGCGCCAAGTCATATAAAGCATCACTGATGAA

### Working with big data
- sequence files may exceed what can be stored in RAM
- working with low-level SimpleFastaParser is often more practical than Bio.SeqIO.parse when working with high-throughput data with an emphasis on speed 



In [3]:
! wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/xenoMrna.fa.gz


--2024-02-21 22:30:47--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/xenoMrna.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6894190900 (6.4G) [application/x-gzip]
Saving to: ‘xenoMrna.fa.gz’

xenoMrna.fa.gz      100%[===================>]   6.42G  13.5MB/s    in 8m 28s  

2024-02-21 22:39:16 (13.0 MB/s) - ‘xenoMrna.fa.gz’ saved [6894190900/6894190900]



In [6]:
# unzip the file 
# testing the performance of SimpleFastaParser vs SeqIO.parse()
from Bio.SeqIO.FastaIO import SimpleFastaParser
import time

start = time.time()
count = 0
xeno_mrna = "/Users/fortino.pinedaveloz/VsCode/FPV_BIX/biopython/xenoMrna.fa"
with open(xeno_mrna) as handle: # parsing using handle == pointer to file
    for seq_id, seq in SimpleFastaParser(handle):
        # seq_id -- sequence identifier
        # seq -- sequence itself
        count += 1
end = time.time()
print(f"time: {end-start}")
print(f"Number of sequences: {count}")

time: 175.9376242160797
Number of sequences: 25855502


In [7]:
start = time.time()
count = 0 
for record in SeqIO.parse(xeno_mrna, "fasta"):
    count += 1
end = time.time()
print(f"time: {end-start}")
print(f"Number of sequences: {count}")

time: 318.87664008140564
Number of sequences: 25855502


- SimpleFastaParser outperforms SeqIO in almost half the time. Although more nested, the performance is worth making this the default method for FASTA parsing 

## Example: Calculate GC content 
GC content is defined as a percentage of symbols in the string 

Equation:
$$ {{ g + c } \over { a + t + g + c }} * 100

In [12]:
from Bio import SeqIO

max_sequence_id = None
max_gc_content = 0
for seq_record in SeqIO.parse("/Users/fortino.pinedaveloz/VsCode/FPV_BIX/biopython/RAD50_gene.fna", "fasta"):
    sequence = str(seq_record.seq) #converts to string 
    sequence_id = seq_record.id
    gc_content = (sequence.count("C") + sequence.count("C")) / len(sequence) * 100
    if gc_content > max_gc_content:
        max_sequence_id = sequence_id
        max_gc_content = gc_content

print(max_sequence_id)
print(max_gc_content)


NC_060929.1:133076780-133166151
38.322964686926554


## Using Biopython to complete transcription 

__Biopython - The Seq Object__
the Seq object is Biopython is python string combined with biological methods


In [13]:
from Bio.Seq import Seq

sequence = Seq("AGTACACTGGT")
print(sequence)

AGTACACTGGT


### Methods

In [14]:
# count() = the Seq object has a .count() method, just like a string
# Note: that this gives a non-overlapping call 
sequence = Seq("AAAA")
sequence.count("AA")

2

In [15]:
# overlapping count, reminiscent of contigs
sequence.count_overlap("AA")

3

In [22]:
# calculate GC content 
from Bio.SeqUtils import gc_fraction
sequence = Seq("AGTCGATCGTAGCGTAGCA")
gc_fraction(sequence)

0.5263157894736842

### Transcription: DNA > RNA

In bio: 
- DNA is double stranded with 5' to 3' being the _coding strand_ and 3' to 5' being the _template_ for transcription

In bioinformatics:
- Transcription can be done by replacing T with U to form RNA product


In [23]:
coding_dna = Seq("AGTCGTTAAGCTATGCGATTCAGTAC")
template_dna = coding_dna.reverse_complement()
template_dna # nucleotide sequences are normally read from the 5' to 3' direction 

Seq('GTACTGAATCGCATAGCTTAACGACT')

In [24]:
# transcribe 
messenger_rna = coding_dna.transcribe()
messenger_rna

Seq('AGUCGUUAAGCUAUGCGAUUCAGUAC')

In [25]:
# The Seq object also includes a back-transcription method for going from the mRNA 
# to the coding strand of the DNA
coding_dna = messenger_rna.back_transcribe()
coding_dna

Seq('AGTCGTTAAGCTATGCGATTCAGTAC')

### Translation: RNA -> protein


In [26]:
messenger_rna.translate() # * == stop codon 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('SR*AMRFS')

In [27]:
# we can also translate directly from the coding DNA strand 
coding_dna.translate()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('SR*AMRFS')